### Pix2Pix modeling

Model repo: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Paper: https://arxiv.org/pdf/1611.07004.pdf

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, img_as_float, img_as_ubyte, io, color
from skimage.measure import compare_ssim
from skimage.measure import compare_mse
from cellvision_lib import get_model_data_splits
# %pylab inline

In [5]:
# Proprocesing the data for pix2pix model
import os
import glob
from shutil import copyfile

MAX_DEPTH = 100
NUM_SAMPLES = 109

# folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized'
# train, test, val = train_test_val(folder_path, channels = 1, train_pp = .67, test_pp = .165, val_pp = .165, set_seed = 1)

# train[0:10]

def clear_test_files(pix2pix_path):
    print('Removing files at ' + pix2pix_path)
    outer_paths = ['A','B']
    inner_paths = ['test','train','val']
    for outer in outer_paths:
        for inner in inner_paths:
            files = glob.glob('{root}/{split}/{inner}/*'.format(root=pix2pix_path, split=outer, inner=inner))
            for f in files:
                os.remove(f)
    print('Done removing files.')
                
def isolate_test_samples(train, val, test):
    num_test = len(test)
    train_new = []
    val_new = []
    test_name = []
    
    for low, ref in train:
        fname = os.path.basename(comp)
        end_sample_prefix_index = fname.find('_channel{}_z'.format(channel)) 
        start_z_index = end_sample_prefix_index + 11
        end_index = fname.find('.tif') 
        sample_prefix = fname[0:end_sample_prefix_index]
                
def setup_images_for_pix2pix(src_path, channel, num_images):
    print("Setting up {} images for channel {}".format(num_images, channel))
    pix_folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel{}_{}'.format(channel,num_images)
    print("At path " + pix_folder_path)
    clear_test_files(pix_folder_path)
    train, test, val = get_model_data_splits(src_path, 
                                      channel = channel, 
                                      train_pp = .67, 
                                      test_pp = .165, 
                                      val_pp = .165, 
                                      seed = 1)

    train_images = train[0:num_images]
    test_images = test[0:num_images]
    val_images = val[0:num_images]
    
    def get_pix_fname(base_dir, ref_fname, _channel, mode):
        fname = os.path.basename(comp)
        end_sample_prefix_index = fname.find('_channel{}_z'.format(channel)) 
        start_z_index = end_sample_prefix_index + 11
        end_index = fname.find('.tif') 
        sample_prefix = fname[0:end_sample_prefix_index]
        z_depth = fname[start_z_index:end_index]
        new_comp_path = '{}/A/{}/{}_channel{}_z{}.jpg'.format(base_dir, mode, sample_prefix, _channel, z_depth)
        new_ref_path = '{}/B/{}/{}_channel{}_z{}.jpg'.format(base_dir, mode, sample_prefix, _channel, z_depth)
        if not os.path.exists('{}/A/{}'.format(base_dir,mode)):
            os.makedirs('{}/A/{}'.format(base_dir,mode))
        if not os.path.exists('{}/B/{}'.format(base_dir,mode)):
            os.makedirs('{}/B/{}'.format(base_dir,mode))
        return new_comp_path, new_ref_path
    
    for i, (comp, ref) in enumerate(train_images):
        new_comp_path, new_ref_path = get_pix_fname(pix_folder_path, ref, channel, "train")
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)
        
    print("done with training images")
    
    for i, (comp, ref) in enumerate(test_images):
        new_comp_path, new_ref_path = get_pix_fname(pix_folder_path, ref, channel, "test")
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)
        
    print("done with testing images")
        
    for i, (comp, ref) in enumerate(val_images):
        new_comp_path, new_ref_path = get_pix_fname(pix_folder_path, ref, channel, "val")
        copyfile(comp, new_comp_path)
        copyfile(ref, new_ref_path)
        
    print("done with validation images")


folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-top50'
# folder_path = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-normalized'

setup_images_for_pix2pix(folder_path, 1, 500)
setup_images_for_pix2pix(folder_path, 2, 500)
setup_images_for_pix2pix(folder_path, 3, 500)
setup_images_for_pix2pix(folder_path, 4, 500)
setup_images_for_pix2pix(folder_path, 5, 500)

# print()
# print(glob.glob('/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1/A/train/*'))
# print()
# print(glob.glob('/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1/B/train/*'))


Setting up 500 images for channel 1
At path /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_500
Removing files at /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_500
Done removing files.
done with training images
done with testing images
done with validation images
Setting up 500 images for channel 2
At path /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_500
Removing files at /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_500
Done removing files.
done with training images
done with testing images
done with validation images
Setting up 500 images for channel 3
At path /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_500
Removing files at /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_500
Done removing files.
done with training images
done with testing images
done with validation images
Setting up 500 images for channel 4
At path /gpfs/data/lionnetlab/cellvision

In [ ]:
"""
Commands to run with the pix2pix framework

python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/testing --num_imgs 200

python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000 --num_imgs 1000
bsub -o fold_1_1000.txt python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_1000 --num_imgs 1000
bsub -o fold_1_1000.txt python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_1000 --num_imgs 1000
bsub -o fold_1_1000.txt python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel4_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel4_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel4_1000 --num_imgs 1000
bsub -o fold_1_1000.txt python datasets/combine_A_and_B.py --fold_A /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel5_1000/A --fold_B /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel5_1000/B --fold_AB /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel5_1000 --num_imgs 1000


bsub -Is -gpu "num=1:mode=exclusive_process:mps=yes" python train.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1 --name cellvision_channel1_100 --model pix2pix --direction AtoB --gpu 0 --display_id 0

bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o jobout.tx python train.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2 --name cellvision_channel2_100 --model pix2pix --direction AtoB --gpu 0 --display_id 0

bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channel1_1000_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000 --name cellvision_channel1_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0
bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channel2_1000_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_1000 --name cellvision_channel2_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0
bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channel3_1000_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_1000 --name cellvision_channel3_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0
bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channel4_1000_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel4_1000 --name cellvision_channel4_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0
bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channel5_1000_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel5_1000 --name cellvision_channel5_1000 --model pix2pix --direction AtoB --gpu 0 --display_id 0

bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o channeltesting_jobout.txt python train.py --save_epoch_freq 25 --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000 --name cellvision_testing --model pix2pix --direction AtoB --gpu 0 --display_id 0


export CHANNEL=4
bsub -gpu "num=1:mode=exclusive_process:mps=yes" -o "$CHANNEL_jobout.txt" python train.py --dataroot "/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel$CHANNEL" --name "cellvision_$CHANNEL_100" --model pix2pix --direction AtoB --gpu 0 --display_id 0

bsub -Is -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000 --name cellvision_channel1_1000 --model pix2pix --direction AtoB --num_test 100
bsub -Is -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel2_1000 --name cellvision_channel2_1000 --model pix2pix --direction AtoB --num_test 100
bsub -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel3_1000 --name cellvision_channel3_1000 --model pix2pix --direction AtoB --num_test 100
bsub -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel4_1000 --name cellvision_channel4_1000 --model pix2pix --direction AtoB --num_test 100
bsub -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel5_1000 --name cellvision_channel5_1000 --model pix2pix --direction AtoB --num_test 100


bsub -Is -gpu "num=1:mode=exclusive_process:mps=yes" python test.py --dataroot /gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1 --name cellvision5 --model pix2pix --direction AtoB

"""

In [ ]:
import glob
import os

channels = list(range(1,6))
sample_sizes =[100, 1000]

result_path = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results'

results_dirs = []
mses = []
ssims = []
for sample_size in sample_sizes:
    sub_results = []
    mses_sub = []
    ssim_sub = []
    for channel in channels:
        print("Getting MSE and SSIM for sample size {} and channel {}".format(sample_size, channel))
        result_dir = '{}/cellvision_channel{}_{}/test_latest/images'.format(result_path, channel, sample_size)
        real_path_low = '{}/*_real_A.png'.format(result_dir)
        real_low_imgs = glob.glob(real_path_low)
        print("getting {} images".format(len(real_low_imgs)))
        channel_mse = []
        channel_ssim = []
        for real_path_low in real_low_imgs:
            fname = os.path.basename(real_path_low)
            fname_prefix = fname[:-11]
            real_path_high = '{}/{}_real_B.png'.format(result_dir, fname_prefix)
            fake_path = '{}/{}_fake_B.png'.format(result_dir, fname_prefix)
            try:
                real_img_high = color.rgb2gray(io.imread(real_path_high).astype(np.uint))
                real_img_low = color.rgb2gray(io.imread(real_path_low).astype(np.uint))
                fake_img = color.rgb2gray(io.imread(fake_path).astype(np.uint))
            except OSError:
                print("OSERROR skipping " + fake_path)
            
            _min = real_img_high.min()
            _max = real_img_high.max()

            low_high_ssim = compare_ssim(real_img_low, real_img_high, data_range=_max-_min)
            fake_high_ssim = compare_ssim(fake_img, real_img_high, data_range=_max-_min)
            channel_ssim.append( (low_high_ssim,fake_high_ssim) )

            low_high_mse = compare_mse(real_img_low, real_img_high)
            fake_high_mse = compare_mse(fake_img, real_img_high)
            channel_mse.append( (low_high_mse,fake_high_mse) )
        
        mses_sub.append(channel_mse)
        ssim_sub.append(channel_ssim)
    mses.append(mses_sub)
    ssims.append(ssim_sub)

print(len(mses[0][0][0]))
# results_dir = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision5/test_latest/images'

# out_imgs = glob.glob(results_dir)


In [ ]:
print(len(ssims))
print(len(ssims[0]))
print(len(ssims[0][0]))
ssims[0][0]

In [ ]:
import statistics

low_means = []
fake_means = []
for sample_size in ssims:
    for channel in sample_size:
        low = [ssim[0] for ssim in channel]
        print(len(low))
        low_std = statistics.stdev(low)
        low_mean = statistics.mean(low)
        print(low_mean)
        low_means.append(low_mean)

        fake = [ssim[1] for ssim in channel]
        fake_std = statistics.stdev(fake)
        fake_mean = statistics.mean(fake)
        fake_means.append(fake_mean)


print("Low ssim mean {} with std {}".format(low_mean,low_std))
print("Fake ssim mean {} with std {}".format(fake_mean,fake_std))

# low_mse = [mse[0] for mse in mses]
# low_std_mse = statistics.stdev(low_mse)
# low_mean_mse = statistics.mean(low_mse)

# fake_mse = [mse[1] for mse in mses]
# fake_std_mse = statistics.stdev(fake_mse)
# fake_mean_mse = statistics.mean(fake_mse)
# print()
# print("Low mse mean {} with std {}".format(low_mean_mse,low_std_mse))
# print("Fake mse mean {} with std {}".format(fake_mean_mse,fake_std_mse))


In [ ]:
low_means

In [ ]:
fake_means

In [ ]:
# Comparing MSE for denoising
plt.plot(list(range(1, 6)), low_means[0:5], label="100 Sample Size No Transformation", color='blue') 
plt.plot(list(range(1, 6)), fake_means[0:5], label="100 Sample Size Prediction", color='red') 
plt.plot(list(range(1, 6)), low_means[5:10], label="1000 Sample Size No Transformation", color='purple') 
plt.plot(list(range(1, 6)), fake_means[5:10], label="1000 Sample Size Prediction", color='green') 

plt.title('Comparing SSIM For Pix2Pix Model')
plt.legend(loc = 'lower right')
# plt.ylim([0, 1])
plt.xlim([1, 5])
plt.axes().set_xticks(list(range(1, 6)), minor=False)

plt.ylabel('SSIM')
plt.xlabel('Channel (larger number -> higher laser intensity)')
plt.show()

In [ ]:
# Showing images
from skimage import exposure
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample112_channel1_z34_real_A.png'
imsk_A = io.imread(path_to_data)
imsk_A = exposure.rescale_intensity(imsk_A,in_range=(0,np.max(imsk_A)))
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample112_channel1_z34_real_B.png'
imsk_B = io.imread(path_to_data)
imsk_B = exposure.rescale_intensity(imsk_B,in_range=(0,np.max(imsk_B)))
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample112_channel1_z34_fake_B.png'
fake = io.imread(path_to_data)
fake = exposure.rescale_intensity(fake,in_range=(0,np.max(fake)))
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(25,5))
# ax[0].imshow(imsk_nl[0]);
ax[0].imshow(imsk_A);
ax[0].axis('off')
ax[1].imshow(imsk_B);
ax[1].axis('off')
ax[2].imshow(fake);
ax[2].axis('off')
plt.show()

In [ ]:
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample112_channel1_z34_real_A.png'
imsk_A = io.imread(path_to_data)
imsk_A = exposure.rescale_intensity(imsk_A,in_range=(0,np.max(imsk_A)))
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample112_channel1_z34_real_B.png'
imsk_B = io.imread(path_to_data)
imsk_B = exposure.rescale_intensity(imsk_B,in_range=(0,np.max(imsk_B)))
path_to_data = '/gpfs/data/lionnetlab/cellvision/pilotdata/20181009-pix2pix/channel1_1000/test/sample61_channel1_z48.jpg'
fake = io.imread(path_to_data)
fake = exposure.rescale_intensity(fake,in_range=(0,np.max(fake)))
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(25,5))
# ax[0].imshow(imsk_nl[0]);
ax[0].imshow(imsk_A);
ax[0].axis('off')
ax[1].imshow(imsk_B);
ax[1].axis('off')
ax[2].imshow(fake);
ax[2].axis('off')
plt.show()

In [ ]:
path_to_data = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample27_channel1_z26_real_A.png'
path_to_data_B = '/home/dg3047/capstone/pytorch-CycleGAN-and-pix2pix/results/cellvision_channel1_1000/test_latest/images/sample27_channel1_z26_real_B.png'

imsk_A = io.imread(path_to_data)
imsk_B = io.imread(path_to_data_B)
_min = imsk_A.min()
_max = imsk_A.max()

low_high_ssim = compare_ssim(imsk_A, real_img_high, data_range=_max-_min)
low_high_ssim